# Coleta de dados complementares no SteamSPY

In [6]:
import json
import os
import requests as rs
import time


url = "https://steamspy.com/api.php"


def GetSteamSpyData(appid,count = 0):
    params = {'request':'appdetails',"appid": appid}
    extraData = {}
    try:
        response = rs.get(url=url, params=params, timeout=60)
        try:
            if response:
                extraData = response.json()
                extraData['spy_status'] = 'Spy_Scrap_Sucess'
            elif response.status_code == 429:
                print(f'Esperando api permitir nova consulta, requisição {count}')
                time.sleep(300)
                extraData = GetSteamSpyData(appid)
                extraData['spy_status'] = 'Spy_Scrap_Sucess'
            else:
                print(response.status_code)
        except Exception as e:
            print(f'{type(e)} : {e}')
            extraData['positive'] = 0
            extraData['negative'] = 0
            extraData['tags'] = 0
            extraData['owners'] = 0
            extraData['spy_status'] = f'Spy_Scrap_Failed {type(e)}'
    except Exception as e:
        print(f'{type(e)} : {e}')
        extraData['positive'] = 0
        extraData['negative'] = 0
        extraData['tags'] = 0
        extraData['owners'] = 0
        extraData['spy_status'] = f'Spy_Scrap_Failed {type(e)}'
    return extraData

In [2]:
import json
from tqdm.notebook import trange, tqdm
import os
import requests as rs
import time

for s in range(0,100):
    if os.path.exists(f'SteamSpyDataAdded/SteamAppData_{s}.json'):
        #print(f'Pacote de dados {s} já foi complementado com dados do SteamSpy')
        continue
    else:
        if os.path.exists(f'RawData/SteamAppData_{s}.json'):
            progresso = tqdm(total=1, desc=f"Progresso do {s}º pacote de dados", position=0, leave=True)
            with open(f'RawData/SteamAppData_{s}.json','r') as file:
                data = json.load(file)

            count = 0 # => Serve para entender e quando a API começa a rejeitar por muito request
            for index,app in enumerate(data):
                if app['scrap_status'] == 'Scrap_Sucess' and app['type'] == 'game':
                    count += 1
                    extraData = GetSteamSpyData(app['steam_appid'],count)
                    data[index]['positive'] = extraData['positive']
                    data[index]['negative'] = extraData['negative']
                    data[index]['tags'] = extraData['tags']
                    data[index]['steamspy_owners'] = extraData['owners']
                    data[index]['spy_status'] = extraData['spy_status']
                progresso.update(1/len(data))
                time.sleep(1)
            with open(f'SteamSpyDataAdded/SteamAppData_{s}.json','w') as file:
                json.dump(data,file,indent=4)
        else:
            print(f'Pacote da dados {s} não encontrado')


Progresso do 98º pacote de dados:   0%|          | 0/1 [00:00<?, ?it/s]

Progresso do 99º pacote de dados:   0%|          | 0/1 [00:00<?, ?it/s]

### Rescrap

In [7]:
import json
from tqdm.notebook import tqdm
import time

def ValidAppToReScrap(app):
    if app['scrap_status'] == 'Scrap_Sucess' and app['type'] == 'game':
        try:
            #Se tiver a string no spy_status quer dizer que falhou anteriormente e portanto é válido uma nova coleta
            return "Spy_Scrap_Failed" in app['spy_status']
        except:
            #Se falhar o Try significa que esse app nunca recebeu os dados do SteamSpy
            return True
    else:
        #Se não é jogo ou não teve dados obtidos da steam não tem pq pegar dados do SPY
        return False

def ReScrapSteamSpy():
    with open('Datasets/SteamDataset.json','r') as file:
        data = json.load(file)

    progress = tqdm(total=1, desc=f"Progresso:", position=0, leave=True)
    for i,app in enumerate(data):
        if ValidAppToReScrap(app):
            extraData = GetSteamSpyData(app['steam_appid'])
            data[i]['positive'] = extraData['positive']
            data[i]['negative'] = extraData['negative']
            data[i]['tags'] = extraData['tags']
            data[i]['steamspy_owners'] = extraData['owners']
            data[i]['spy_status'] = extraData['spy_status']
            time.sleep(1)
        progress.update(1/len(data))
    with open('Datasets/SteamDataset_w_Spy.json','w') as file:
        json.dump(data,file,indent=4)

ReScrapSteamSpy()


Progresso::   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:

def SetSpyScrapSucess(seg):
    with open(f'SteamSpyDataAdded/SteamAppData_{seg}.json','r') as file:
        data = json.load(file)
    for i,app in enumerate(data):
        print(data[i])
        if app['scrap_status'] == 'Scrap_Sucess' and app['type'] == 'game':
            data[i]['spy_status'] = 'Spy_Scrap_Sucess'
        #else:
        #    data[i].pop('spy_status')

    with open(f'SteamSpyDataAdded/SteamAppData_{seg}.json','w') as file:
       json.dump(data,file,indent=4)

def UpdateSegElement(seg, appid):
    with open(f'SteamSpyDataAdded/SteamAppData_{seg}.json','r') as file:
        data = json.load(file)

    extraData = GetSteamSpyData(appid,0)
    index = list(filter(lambda a: a[1].get('steam_appid') == appid, enumerate(data)))[0][0]
    print(index)

    data[index]['positive'] = extraData['positive']
    data[index]['negative'] = extraData['negative']
    data[index]['tags'] = extraData['tags']
    data[index]['steamspy_owners'] = extraData['owners']
    data[index]['spy_status'] = extraData['spy_status']

    #print(data[index])

    with open(f'SteamSpyDataAdded/SteamAppData_{seg}.json','w') as file:
        json.dump(data,file,indent=4)

#UpdateSegElement(3,2512930)

#SetSpyScrapSucess(0)
#SetSpyScrapSucess(1)
#SetSpyScrapSucess(2)
#SetSpyScrapSucess(3)
#SetSpyScrapSucess(4)

In [11]:
import json
from tqdm.notebook import trange, tqdm
import os
import requests as rs


url = "https://steamspy.com/api.php"
params = {'request':'appdetails',"appid": 949230}

response = rs.get(url=url, params=params, timeout=10)
if response:
    extraData = response.json()
    print(extraData)
    for e in extraData:
        print(e)
else:
    print(response.status_code)

{'appid': 949230, 'name': 'Cities: Skylines II', 'developer': 'Colossal Order Ltd.', 'publisher': 'Paradox Interactive', 'score_rank': '', 'positive': 13631, 'negative': 10610, 'userscore': 0, 'owners': '500,000 .. 1,000,000', 'average_forever': 0, 'average_2weeks': 0, 'median_forever': 0, 'median_2weeks': 0, 'price': '4999', 'initialprice': '4999', 'discount': '0', 'ccu': 49182, 'languages': 'English, French, Italian, German, Japanese, Korean, Polish, Russian, Simplified Chinese, Portuguese - Brazil, Traditional Chinese, Spanish - Spain', 'genre': 'Simulation', 'tags': {'Simulation': 364, 'City Builder': 346, 'Building': 267, 'Singleplayer': 228, 'Sandbox': 209, 'Management': 199, 'Economy': 189, 'Strategy': 186, 'Realistic': 164, 'Multiplayer': 129, 'Moddable': 92, 'Relaxing': 68, 'Casual': 64, 'Resource Management': 57, 'Atmospheric': 55, 'Open World': 23, 'Great Soundtrack': 20, 'Psychological Horror': 10, 'Early Access': 10, 'Transportation': 6}}
appid
name
developer
publisher
sco

Existe um grau de discrepância na quantidade de avaliações da página steam e oq aparece no steamspy, será q essa discrepância está presente também no dado extraído via API?

De fato existe essa discrepância, ao que tudo indica a steam só retorna parte das avaliações (podendo ser um filtro de avaliações válidas ou algo do tipo) enquanto o steamSpy retorna todas as avaliações sem filtro

Essa teoria foi elaborada comparando o 1886260 e o 435150 com os dados da steam e do steamspy
- O 1886260 tem 1 avaliação na página steam, contudo, a API da steam não retorna nenhum valor de recommendation
- Já no SteamSPy consta para o 1886260 1 avaliação positiva

In [15]:
import json
from tqdm.notebook import trange, tqdm
import os
import requests as rs
      
            
for s in range(0,100):
    if os.path.exists(f'RawData/SteamAppData_{s}.json'):
        with open(f'RawData/SteamAppData_{s}.json','r') as file:
            data = json.load(file)
            tGames = len(list(filter(lambda a: a.get('scrap_status') == 'Scrap_Sucess' and a.get('type') == 'game', data)))
            print(f'Total de jogos no pacote {s}: {tGames}')

Total de jogos no pacote 0: 990


Jogos novos e/ou não lançados e/ou muito pouco populares podem vir sem os dados de Tag

- Em 2472840 foi observado ausência de dados de tag, recomendações/avaliações, o motivo até agora é só especulação, pois esses dados existem na página steam

Nesses casos de ausência de dados via APIs o que restá é um webscrapping de front page, o que pode ser um número mais reduzido (esperamos que seja) pdoe ser viável